# Creación de RDD

#### [Introducción a Spark con Python, por Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

En este notebook introduciremos dos formas diferentes de cargar datos en la estructura básica de Spark, el **Resilient Distributed Dataset** o **RDD**. Un RDD es una colección distribuida de elementos. Todo el trabajo en Spark se expresa como la creación de nuevos RDDs, la transformación de RDDs existentes, o la ejecución de acciones sobre RDDs para calcular un resultado. Spark distribuye automáticamente los datos contenidos en los RDDs a través de tu clúster y paraleliza las operaciones que realizas sobre ellos.

#### Referencias

El libro de referencia para estos y otros temas relacionados con Spark es *Learning Spark* de Holden Karau, Andy Konwinski, Patrick Wendell, y Matei Zaharia.

El conjunto de datos de la competición KDD Cup 1999 está descrito en detalle [aquí](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99).

## SparkContext

In [ ]:
# Importamos la librería pyspark para trabajar con Spark
import pyspark

# Obtenemos el SparkContext desde la SparkSession existente
# El SparkContext es el punto de entrada principal para la funcionalidad de Spark
sc = spark.sparkContext # spark es la SparkSession

In [ ]:
# Mostramos el SparkContext para verificar que está activo
sc

## Obteniendo los archivos de datos

En este notebook usaremos el conjunto de datos reducido (10 por ciento) proporcionado para la KDD Cup 1999, que contiene casi medio millón de interacciones de red. El archivo se proporciona como un archivo *Gzip* que descargaremos localmente.

In [ ]:
# Definimos la URL del archivo de datos que queremos descargar
url = "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz"

# Importamos SparkFiles para gestionar archivos distribuidos
from pyspark import SparkFiles

# Añadimos el archivo al contexto de Spark para que esté disponible en todos los nodos
spark.sparkContext.addFile(url)

In [ ]:
# Creamos un RDD leyendo el archivo de texto comprimido
# SparkFiles.get() obtiene la ruta local del archivo descargado
# textFile() puede manejar archivos comprimidos directamente
myRDD = sc.textFile("file://" + SparkFiles.get("kddcup.data_10_percent.gz"))

In [ ]:
# Verificamos la clase SparkFiles
SparkFiles

In [ ]:
# Mostramos los archivos disponibles en el sistema de archivos de Databricks
display(dbutils.fs.ls("/databricks-datasets/COVID/USAFacts/"))

In [ ]:
# Tomamos las primeras 5 líneas del RDD para visualizar los datos
# take() es una acción que devuelve los primeros n elementos
myRDD.take(5)

## Creando un RDD desde un archivo

La forma más común de crear un RDD es cargarlo desde un archivo. Observa que el método `textFile` de Spark puede manejar archivos comprimidos directamente.

In [ ]:
# Verificamos el tipo de objeto que hemos creado
type(myRDD)

Ahora tenemos nuestro archivo de datos cargado en el RDD `raw_data`.

Sin entrar en las *transformaciones* y *acciones* de Spark, lo más básico que podemos hacer para verificar que obtuvimos el contenido correcto del RDD es usar `count()` para contar el número de líneas cargadas desde el archivo al RDD.

In [ ]:
# Contamos el número total de filas en el RDD
# count() es una acción que devuelve el número de elementos
myRDD.count() # Cantidad de filas

También podemos verificar las primeras entradas en nuestros datos.

In [ ]:
# Extraemos las primeras 2 líneas del RDD
myRDD.take(2)

En los siguientes notebooks, usaremos estos datos crudos para aprender sobre las diferentes transformaciones y acciones de Spark.

## Creando un RDD usando `parallelize`

Otra forma de crear un RDD es paralelizar una lista ya existente.

In [ ]:
# Creamos un rango de 100 números (0 a 99)
a = range(100)

# Paralelizamos el rango para crear un RDD distribuido
# Cuando no especificamos el número de particiones, depende de los cores disponibles
data = sc.parallelize(a) # leer del range y me va a devolver un RDD, ¿cuántas particiones? Cuando no digo nada, depende de los cores

# Verificamos el tipo de objeto creado
type(data) # todo esto es lazy (perezoso), se lo ha apuntado pero no se ha ejecutado

Como hicimos antes, podemos usar `count()` para contar el número de elementos en el RDD.

In [ ]:
# Contamos los elementos en el RDD
data.count()

Como antes, podemos acceder a los primeros elementos de nuestro RDD.

In [ ]:
# Obtenemos los primeros 5 elementos del RDD
data.take(5)

## Obtener datos y particiones

In [ ]:
# Recopilamos todos los datos del RDD al nodo driver
# ¡CUIDADO! collect() trae TODOS los datos a memoria, puede ser peligroso con datasets grandes
rddCollect = data.collect() # data RDD (Resilient Distributed Dataset) es una estructura. Con collect lee todas las particiones y se meten en el nodo driver, y se puede dañar

# Mostramos el número de particiones en las que está dividido el RDD
print("Number of Partitions: " + str(data.getNumPartitions()))

# Obtenemos el primer elemento del RDD (es una acción)
print("Action: First element: " + str(data.first()))

# Imprimimos todos los datos recopilados
print(rddCollect)

# Nota: Para especificar el número de particiones:
# data = sc.parallelize(a, p) # hace p particiones